In [1]:
import FinanceDataReader as fdr
import requests
import pandas as pd
import time
import ta
import openpyxl
import warnings
import cloudscraper
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

import datetime
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from defs_pred_auto import Data_Scrap_Pred
from datetime import datetime, timedelta
from pykrx import stock
from pykrx import bond
from PublicDataReader import Fred
from functools import reduce
from collections import OrderedDict
from defs_pred import Marcap, filter_df
from io import BytesIO
from openpyxl.utils.dataframe import dataframe_to_rows
from ics import Calendar, Event

[main 4b7bf75] Commit message
 2 files changed, 21 insertions(+), 22 deletions(-)


error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/moonseunghoo/Stock_Machine_Learning.git'


CompletedProcess(args=['git', 'push', '-u', 'origin', 'master'], returncode=1)